In [ ]:
import os
import pandas as pd
import numpy as np
import math
import pybv

In [ ]:
def _to_brain_vision(the_data):
    the_data_path = os.path.join(the_drive,os.sep,the_repo,the_subrepo,the_type, the_data)
    the_data_slicing = os.path.join(the_drive,os.sep,the_repo,the_subrepo,the_type, the_data_slice)
    the_data_raw = os.path.join(the_drive,os.sep,the_repo,the_subrepo,the_type,  data_raw)
    ch_layout = os.path.join(the_drive,os.sep,the_repo,the_subrepo,the_type,  ch_config)

    # Slicing 
    df_slice = pd.read_csv(the_data_slicing)
    df_slice['f'] = df_slice['Photodiode-ch1']
    # Extracting peak value 
    a = df_slice[df_slice.f >= peak_value]   
    a.reset_index(level=0, inplace=True)
    value = a.iloc[0, 0]
    initial_corrected_data = df_slice.iloc[value:,0:5]
    
    df = pd.read_csv(the_data_path)
    df.rename(columns={'rotation': 'event_label'}, inplace=True)
    event_label = df['event_label'].dropna()
    
    df_raw = pd.read_csv(the_data_raw)
    result_EEG = df_raw.iloc[value:,:]
    file_name = the_type + ".csv"
    result_EEG.to_csv(file_name, sep=',', index=False, encoding='utf-8')
    # event sequnecing and labeling w.r.t trials
    x = []
    for trial_number in range(1, 6):
        trial_start = int((rest * sfreq) + x[-1]) if trial_number > 1 else event_start
        trial_length = int(trial_start + stim * sfreq * n_stim)
        for i in range(trial_start, trial_length + 1, stim_len):
            x.append(i)
    events = pd.DataFrame({'event_sequence': x, 'event_label': event_label})
    events['trial'] = np.repeat(np.arange(1, 6), n_stim)
    events.to_csv(the_event,sep=',', index=False, encoding='utf-8')   
    
    # Data to brain vision format
    df_mne = result_EEG
    df_mne = df_mne.drop(['timestamp', 'sequence', 'battery', 'flags'], axis=1)
    df_mne["EEG"] = df_mne['EEG-ch1'] 
    layout_raw = pd.read_excel(ch_layout, index_col=0)
    ch_names = []
    eeg_cols = [col for col in df_mne.columns if 'EEG' in col]
    for i in range(len(eeg_cols)):
        the_name = layout_raw.index[layout_raw['Channel number'] == i+1].tolist()[0].replace('0','O')
        ch_names.append(the_name)

    data = df_mne.to_numpy().transpose()
    data = np.vstack(data/1e6)
    ch_types = ['eeg'] * 32 + ['eog'] * 1
    unit= ['µV'] * 33
    event = pd.read_csv(the_event)
    events_int = np.array(events).astype(int)

    pybv.write_brainvision(data=data, sfreq=sfreq, ch_names=ch_names,
                  folder_out='./', 
                  fname_base=the_type, events=events_int,
                  unit=unit, overwrite=True)     
        
    

if __name__ == '__main__':
    data_name = "04466_S021_T002_rsvp_paradigm_2023_Jul_27_1106"
    layout = "Layout"
    data_slice = "Photodiode"
    the_raw_data = "EEG"
    the_drive = "D:"
    the_repo = "data"
    the_subrepo = "data_testing"
    the_type = "04466_S021_T002"
    the_event = "04466_S021_T002_events" + ".csv"
    the_id = f"{data_name.split('_')[0]}"
    
    
    the_data = f"{data_name}.csv"
    the_data_slice = f"{data_slice}.csv"
    data_raw=f"{the_raw_data}.csv"
    ch_config= f"{layout}.xlsx"
    event_start = int(256 * 17.5)  # sampling frequency * time until the start of first trial
    sfreq=256                     # sampling frequency
    stim = 0.3                   # token display in seconds
    n_stim = 300                # number of tokens per trial
    stim_len = math.ceil(sfreq * stim)     
    rest=10                    # Rest betweeen the trials in seconds
    peak_value = 160000       # peak value depend on visual inspection of photodiode cutoff value for green screen
    _to_brain_vision(the_data)